In [1]:


import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'


config = tf.ConfigProto()
config.gpu_options.allow_growth=True

#check TensorRT version
!dpkg -l | grep nvinfer



ii  libnvinfer-bin                         6.0.1-1+cuda10.1                  amd64        TensorRT binaries
ii  libnvinfer-dev                         6.0.1-1+cuda10.1                  amd64        TensorRT development libraries and headers
ii  libnvinfer-plugin-dev                  6.0.1-1+cuda10.1                  amd64        TensorRT plugin libraries
ii  libnvinfer-plugin6                     6.0.1-1+cuda10.1                  amd64        TensorRT plugin libraries
ii  libnvinfer6                            6.0.1-1+cuda10.1                  amd64        TensorRT runtime libraries
ii  python3-libnvinfer                     6.0.1-1+cuda10.1                  amd64        Python 3 bindings for TensorRT
ii  python3-libnvinfer-dev                 6.0.1-1+cuda10.1                  amd64        Python 3 development package for TensorRT


In [14]:
def load_graph(model_file):
  graph = tf.Graph()
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  with graph.as_default():
    tf.import_graph_def(graph_def)

  return graph

def load_graph_def(model_file):  
  graph_def = tf.GraphDef()

  with open(model_file, "rb") as f:
    graph_def.ParseFromString(f.read())
  
  return graph_def

In [3]:
def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
  input_name = "file_reader"
  output_name = "normalized"
  file_reader = tf.read_file(file_name, input_name)
  if file_name.endswith(".png"):
    image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
  elif file_name.endswith(".gif"):
    image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
  elif file_name.endswith(".bmp"):
    image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
  else:
    image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")
  float_caster = tf.cast(image_reader, tf.float32)
  dims_expander = tf.expand_dims(float_caster, 0)
  resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
  normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
  sess = tf.compat.v1.Session()
  result = sess.run(normalized)

  return result

In [4]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [23]:
  file_name = "./data/grace_hopper.jpg"
  model_file = \
    "./data/inception_v3_2016_08_28_frozen.pb"
  label_file = "./data/imagenet_slim_labels.txt"
  input_height = 299
  input_width = 299
  input_mean = 0
  input_std = 255
  input_layer = "input"
  output_layer = "InceptionV3/Predictions/Reshape_1"

In [24]:
graph = load_graph(model_file)
    
t = read_tensor_from_image_file(
      file_name,
      input_height=input_height,
      input_width=input_width,
      input_mean=input_mean,
      input_std=input_std)


In [25]:
  input_name = "import/" + input_layer
  output_name = "import/" + output_layer
  input_operation = graph.get_operation_by_name(input_name)
  output_operation = graph.get_operation_by_name(output_name)

  with tf.compat.v1.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
  results = np.squeeze(results)

  top_k = results.argsort()[-5:][::-1]
  labels = load_labels(label_file)
  for i in top_k:
    print(labels[i], results[i])

military uniform 0.8343052
mortarboard 0.021869471
academic gown 0.010358049
pickelhaube 0.00800826
bulletproof vest 0.0053509204


In [26]:
BATCH_SIZE = 1

graph_def = load_graph_def(model_file)
    
trt_fp32_graph = trt.create_inference_graph(
    input_graph_def=graph_def,
    outputs=['InceptionV3/Predictions/Reshape_1'],
    max_batch_size=BATCH_SIZE,
    precision_mode="FP32")



In [31]:
with tf.gfile.GFile('./data/inception_v3_2016_08_28_frozen_tftrt_fp32.pb', 'wb') as f:
    f.write(trt_fp32_graph.SerializeToString())

In [33]:
!ls -l data

total 282868
-rwxrwxrwx 1 root   root     61306 Nov  8 22:40 grace_hopper.jpg
-rwxrwxrwx 1 270850 5000     10479 Jan 18  2017 imagenet_slim_labels.txt
-rwxrwxrwx 1 270850 5000  95746802 Jan 18  2017 inception_v3_2016_08_28_frozen.pb
-rw-r--r-- 1 root   root 193832798 Nov 12 16:15 inception_v3_2016_08_28_frozen_tftrt_fp32.pb


In [34]:
  graph = load_graph('./data/inception_v3_2016_08_28_frozen_tftrt_fp32.pb')

  input_name = "import/" + input_layer
  output_name = "import/" + output_layer
  input_operation = graph.get_operation_by_name(input_name)
  output_operation = graph.get_operation_by_name(output_name)

  with tf.compat.v1.Session(graph=graph) as sess:
    results = sess.run(output_operation.outputs[0], {
        input_operation.outputs[0]: t
    })
  results = np.squeeze(results)

  top_k = results.argsort()[-5:][::-1]
  labels = load_labels(label_file)
  for i in top_k:
    print(labels[i], results[i])

military uniform 0.8343062
mortarboard 0.02186937
academic gown 0.010358016
pickelhaube 0.008008178
bulletproof vest 0.005350883
